In [1]:
%matplotlib widget

import tensorflow as tf
import os
import sys
import cv2
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import random

from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import GlobalMaxPooling2D
from numpy.linalg import norm
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from annoy import AnnoyIndex

In [2]:
DATASET_PATH = "/Users/jeremy/Google Drive/datasets/fashion-dataset/"
print(os.listdir(DATASET_PATH))

['embeddings.tsv', '.DS_Store', 'images.csv', 'images', 'styles', 'styles.csv', '.ipynb_checkpoints', 'embeddings.csv', 'resnet50-embeddings.pkl']


In [3]:
df = pd.read_csv(DATASET_PATH + "styles.csv", nrows=5000, error_bad_lines=False)
df['image'] = df.apply(lambda row: str(row['id']) + ".jpg", axis=1)
df = df.reset_index(drop=True)
df.head(10)

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,39386.jpg
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,59263.jpg
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants,21379.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt,53759.jpg
5,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,2011,Casual,Inkfruit Mens Chain Reaction T-shirt,1855.jpg
6,30805,Men,Apparel,Topwear,Shirts,Green,Summer,2012,Ethnic,Fabindia Men Striped Green Shirt,30805.jpg
7,26960,Women,Apparel,Topwear,Shirts,Purple,Summer,2012,Casual,Jealous 21 Women Purple Shirt,26960.jpg
8,29114,Men,Accessories,Socks,Socks,Navy Blue,Summer,2012,Casual,Puma Men Pack of 3 Socks,29114.jpg
9,30039,Men,Accessories,Watches,Watches,Black,Winter,2016,Casual,Skagen Men Black Watch,30039.jpg


In [4]:
def get_img_path(img):
  return DATASET_PATH + "images/" + img

In [5]:
df_embs = pickle.load(open(DATASET_PATH + "resnet50-embeddings.pkl", "rb"))

In [6]:
# Filter only embeddings with master category 'Apparel' and save indices to filter for them later
df_filtered = df.loc[df.masterCategory == 'Apparel']
df_filtered_idx = df_filtered.index.values.tolist()
df_filtered = df_filtered.reset_index(drop=True)
df_filtered.masterCategory.unique(), df_filtered.shape, len(df_filtered_idx)

(array(['Apparel'], dtype=object), (2297, 11), 2297)

In [7]:
embs_filtered = []
for i, row in df_embs.iterrows():
    if i in  df_filtered_idx:
        embs_filtered.append(row)
df_embs_filtered = pd.DataFrame(embs_filtered)
df_embs_filtered = df_embs_filtered.reset_index(drop=True)
df_embs_filtered

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,5.661316,2.366165,0.000000,3.255730,0.845208,2.673841,8.780870,6.280786,0.787188,1.587657,...,2.463610,0.000000,1.825994,20.485268,3.902863,0.000000,8.225969,1.313863,0.000000,10.820526
1,5.790417,9.593648,0.000000,9.431670,0.287235,0.259945,17.028748,3.037453,0.136341,0.000000,...,1.670570,0.013891,10.338681,4.784722,0.189140,0.000000,7.739144,0.372693,0.000000,8.903802
2,5.882203,1.958606,0.000000,12.519882,0.000000,0.000000,9.383594,3.781614,0.000000,1.490617,...,17.825478,0.000000,3.592507,2.042305,0.000000,0.000000,6.656779,1.457314,1.694204,1.403327
3,0.245968,17.383430,0.493824,3.374468,2.567368,0.000000,6.482537,5.862639,0.000000,3.164866,...,4.464024,0.447075,1.448143,12.884138,0.000000,0.397501,5.028075,1.939697,0.000000,11.485400
4,0.028546,18.567099,0.000000,0.757250,1.504949,0.000000,1.982024,2.963777,1.072765,4.539300,...,3.247082,4.273134,3.088643,16.997053,0.000000,0.000000,2.258095,1.595798,0.000000,7.674047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2292,3.134137,12.337814,3.281830,0.000000,6.123401,6.273666,3.345068,4.230134,2.082114,2.691854,...,5.392367,2.139013,7.540308,5.394151,6.070920,0.000000,0.000000,0.000000,0.000000,10.168349
2293,2.800386,19.866081,3.215006,2.487933,0.147357,0.000000,7.958175,5.669247,0.706661,4.877097,...,10.292748,0.000000,0.738173,16.109261,0.383140,0.000000,4.432935,2.898650,0.416252,12.830476
2294,4.801172,16.536064,2.132150,2.630906,1.190370,0.243107,1.454644,3.500423,0.627453,5.221931,...,0.000000,1.886256,3.294083,17.186272,0.000000,2.014549,1.663332,2.118907,0.000000,6.815829
2295,3.184294,14.538852,6.306130,0.000000,3.361131,6.701039,0.729837,0.295319,0.252367,3.175583,...,0.000000,1.471544,1.482233,22.654318,2.425646,12.427614,8.393599,0.000000,9.616654,3.375090


In [8]:
# Perform PCA over the embeddings to reduce dimensionality
num_feature_dimensions = 100  # Set the number of embedding dimensions
pca = PCA(n_components = num_feature_dimensions)
embs_compressed = pca.fit_transform(df_embs_filtered)
df_embs_filtered_compressed = pd.DataFrame(embs_compressed)
df_embs_filtered_compressed

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-64.281143,-0.382205,10.129696,-22.772249,-38.307953,10.126498,36.410583,-46.646549,-34.147324,8.802656,...,2.473552,0.815058,-1.324206,6.417312,3.944019,2.259099,-4.700742,8.086054,-0.300124,-1.886420
1,-50.859028,-78.791443,-48.603302,-5.542873,57.489609,-30.835249,-2.337715,-15.355863,-7.670314,-7.935178,...,-0.244214,6.119100,-2.582830,-6.156396,1.553834,5.651353,9.528061,-4.089435,7.066453,-6.111871
2,-21.894361,-69.084808,-56.701500,-13.024251,48.613518,-35.151257,6.533604,12.986296,1.046202,-23.648275,...,1.546429,-1.480428,-13.130416,8.454337,3.512072,-1.920022,5.721154,7.489941,5.329553,-1.222036
3,-52.814491,-30.695797,24.538797,18.532673,-15.459093,-4.578689,-20.258471,5.052539,-18.399326,14.211835,...,0.487577,5.164029,-6.627130,-7.596750,-1.990091,-1.946972,5.223144,8.517446,2.530492,5.019147
4,-26.194769,-12.308703,56.018478,28.274584,4.040716,0.309535,20.197052,11.746017,-15.117895,0.661892,...,-1.006946,2.984005,-1.661335,6.169484,0.039874,-6.870212,4.883084,1.605334,4.553557,3.243586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2292,99.575600,-30.713911,-20.068457,-25.830212,-22.531929,51.434818,-40.079025,-24.682705,-38.467987,53.920879,...,-7.343183,-8.435580,-10.091974,4.403514,2.639768,14.398014,3.600716,-2.999894,-10.551106,1.857587
2293,-26.236425,-12.102087,52.941200,21.009840,-2.619732,-23.604729,3.913823,4.513413,-3.499752,-43.001534,...,10.254016,5.801165,-2.908997,7.767170,-4.840563,-2.381317,-5.673163,-6.208100,4.520116,-8.425411
2294,-29.318752,36.647446,31.197523,14.334846,26.817392,-5.634312,-21.891565,-2.816176,-21.250431,2.923815,...,3.307360,-3.007015,1.464625,11.379956,-1.496132,4.083375,-9.034772,6.651826,5.763023,1.647731
2295,54.131592,63.903698,-8.761909,-63.546356,17.439548,9.300051,16.524513,42.935131,53.400990,43.313881,...,13.089308,-3.970719,-6.844429,-4.788764,4.900023,9.523685,7.353218,10.253664,-6.021593,0.498545


In [9]:
# Map season values to either Spring or Winter
for i, row in df_filtered.iterrows():
    if df_filtered.at[i, 'season'] == "Spring":
        df_filtered.at[i, 'season'] = "Summer"
    elif df_filtered.at[i, 'season'] == "Fall":
        df_filtered.at[i, 'season'] = "Winter"
df_filtered.season.unique()

array(['Winter', 'Summer'], dtype=object)

In [9]:
# Application of SVM to create the axis for a given feature
X = df_embs_filtered_compressed
y = df_filtered["season"]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
svm_clf = LinearSVC(C=1, max_iter=10000000)
svm_clf.fit(X_scaled, y)

LinearSVC(C=1, max_iter=10000000)

In [10]:
number_of_embeddings = df_filtered.shape[0]

f = 100
t = AnnoyIndex(f, 'euclidean')
for index, row in df_embs_filtered_compressed.iterrows():
    v = row.iloc[row.index].values.tolist()
    t.add_item(index, v)

t.build(10)
t.save('annoy_index_100d.ann')

True

In [28]:
get_nearest_neighbour(user_position, df_filtered)

15785

In [22]:
def get_nearest_neighbour(user_position, df):
   i = t.get_nns_by_vector(user_position, 1)
   nearest_neighbour = df.iloc[i]['id'].values[0]
   return nearest_neighbour

In [12]:
def annotate_nearest_neighbour(nearest_neighbour, df):
    arr_img = plt.imread(get_img_path(df.loc[df['id'] == nearest_neighbour].image.values[0]))
    plt.imshow(arr_img)

In [13]:
def compute_navigation_axis(emb, w, range):
    nav_axis = []
    for i, feature in enumerate(emb):
        feature_val = emb[feature].values[0]
        if w[i] > 0:
            pos_range = np.arange(start=feature_val, stop=feature_val + range, step=w[i])
            neg_range = np.arange(start=feature_val, stop=feature_val - range, step=-w[i])
            nav_axis.append(np.sort(np.concatenate((pos_range, neg_range))))
        else:
            pos_range = np.arange(start=feature_val, stop=feature_val + range, step=-w[i])
            neg_range = np.arange(start=feature_val, stop=feature_val - range, step=w[i])
            nav_axis.append(np.sort(np.concatenate((pos_range, neg_range))))
    return nav_axis

In [14]:
def initialize_user_position(axis, idx):
    user_position = []
    for dim in axis:
        user_position.append(dim[idx])
    return user_position

In [35]:
def update_user_position(change):
    """Update the user position after the slider value has changed"""
    idx = np.where(np.isclose(nav_axis[0], min(nav_axis[0], key=lambda x:abs(x-change.new))))[0][0]
    new_user_pos = []
    for i in range(len(nav_axis)):
        new_user_pos.append(nav_axis[i][idx])
    user_position = new_user_pos
    nearest_neighbour = get_nearest_neighbour(user_position, df_filtered)
    annotate_nearest_neighbour(nearest_neighbour, df_filtered)
    plt.title('Nearest Embedding: {} with season: {}'.format(nearest_neighbour, df_filtered.loc[df_filtered['id'] == nearest_neighbour].season.values[0]))
    fig.canvas.draw()
    fig.canvas.flush_events()

In [55]:
from ipywidgets import AppLayout, FloatSlider

# Create matplotlib figure for displaying fashion items
fig = plt.figure(figsize=(7, 7))

# Create orthogonal navigation axis
w = svm_clf.coef_[0]
rand_emb = df_embs_filtered_compressed.sample()
nav_axis = compute_navigation_axis(rand_emb, w, 100)

min_axis_len = 100000
for i in range(len(nav_axis)):
    if len(nav_axis[i]) < min_axis_len:
        min_axis_len = len(nav_axis[i])
min_axis_len

starting_idx = np.where(np.isclose(nav_axis[0], min(nav_axis[0], key=lambda x:abs(x-rand_emb[0].values[0]))))[0][0]
user_position = initialize_user_position(nav_axis, starting_idx)

nearest_neighbour = get_nearest_neighbour(user_position, df_filtered)
annotate_nearest_neighbour(nearest_neighbour, df_filtered)

plt.title('Nearest Embedding: {} with season: {}, user_pos:'.format(nearest_neighbour, df_filtered.loc[df_filtered['id'] == nearest_neighbour].season.values[0], user_position[0]))

# Create Slider to navigate in embedding space
slider = FloatSlider(
    orientation="horizontal",
    description="x-Position:",
    value=user_position[0],
    min=min(nav_axis[0][:min_axis_len]),
    max=max(nav_axis[0][:min_axis_len])
)
slider.layout.margin = '0px 10% 0px 10%'
slider.layout.width = '40%'

slider.observe(update_user_position, names='value')

display(slider)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

FloatSlider(value=-12.079179763793945, description='x-Position:', layout=Layout(margin='0px 10% 0px 10%', widt…